This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# Install python-dotenv
!pip install python-dotenv

In [2]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

o8EkCziA1xWLoKCxKYE8


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [3]:
# First, import the relevant modules
import requests
import json

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [4]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = f'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?api_key={API_KEY}'
response = requests.get(url)
response_json = response.json()

In [31]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

#First Level

keys = response_json['dataset_data'].keys()
for k in keys:
    item = response_json['dataset_data'][k]
    if isinstance(item, list) or isinstance(item, dict):
        # It's a nested structure if it's a list or a dictionary
        print(f'length {k}: {len(item)} (Nested level detected)')
    else:
        # It's not a nested structure
        print(f'length {k}: No nested level. Type: {type(item).__name__}')
        
'''From the documentation and the last code we can say that the first level of the json contains informational data
about the dataset (metadata) the most important collections are the column names as well as data that would be the third 
second nested level. we should use the combination of this two fields to create a new dataset.'''

length limit: No nested level. Type: NoneType
length transform: No nested level. Type: NoneType
length column_index: No nested level. Type: NoneType
length column_names: 11 (Nested level detected)
length start_date: No nested level. Type: str
length end_date: No nested level. Type: str
length frequency: No nested level. Type: str
length data: 5268 (Nested level detected)
length collapse: No nested level. Type: NoneType
length order: No nested level. Type: NoneType


'From the documentation and the last code we can say that the first level of the json contains informational data\nabout the dataset (metadata) the most important collections are the column names as well as data that would be the third \nsecond nexted level. we should use the combination of this two fields to create a new dataset.'

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

**1. Year 2017**

In [7]:
data = response_json['dataset_data']['data']

# Filter for entries from the year 2017
data_2017 = [entry for entry in data if entry[0].startswith('2017')]
print(data_2017[:5])

[['2017-12-29', 51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None], ['2017-12-28', 51.65, 51.82, 51.43, 51.6, None, 40660.0, 2099024.0, None, None, None], ['2017-12-27', 51.45, 51.89, 50.76, 51.82, None, 57452.0, 2957018.0, None, None, None], ['2017-12-22', 51.05, 51.5, 50.92, 51.32, None, 71165.0, 3641949.0, None, None, None], ['2017-12-21', 51.16, 51.52, 50.9, 51.4, None, 120649.0, 6179433.0, None, None, None]]


**2. Python to Dict**

In [8]:
'''
The name of the dataset is dataset_date and we need the column names to make the data underestandable as well as 
each data point data.

so lets mixed it together.
 ''' 
column_names = response_json['dataset_data']['column_names']
formatted_data = []

formatted_data = [{column: value for column, value in zip(column_names, day_data)} for day_data in data_2017]
    
print(formatted_data[:5])

[{'Date': '2017-12-29', 'Open': 51.76, 'High': 51.94, 'Low': 51.45, 'Close': 51.76, 'Change': None, 'Traded Volume': 34640.0, 'Turnover': 1792304.0, 'Last Price of the Day': None, 'Daily Traded Units': None, 'Daily Turnover': None}, {'Date': '2017-12-28', 'Open': 51.65, 'High': 51.82, 'Low': 51.43, 'Close': 51.6, 'Change': None, 'Traded Volume': 40660.0, 'Turnover': 2099024.0, 'Last Price of the Day': None, 'Daily Traded Units': None, 'Daily Turnover': None}, {'Date': '2017-12-27', 'Open': 51.45, 'High': 51.89, 'Low': 50.76, 'Close': 51.82, 'Change': None, 'Traded Volume': 57452.0, 'Turnover': 2957018.0, 'Last Price of the Day': None, 'Daily Traded Units': None, 'Daily Turnover': None}, {'Date': '2017-12-22', 'Open': 51.05, 'High': 51.5, 'Low': 50.92, 'Close': 51.32, 'Change': None, 'Traded Volume': 71165.0, 'Turnover': 3641949.0, 'Last Price of the Day': None, 'Daily Traded Units': None, 'Daily Turnover': None}, {'Date': '2017-12-21', 'Open': 51.16, 'High': 51.52, 'Low': 50.9, 'Close'

**3. Lowest and Highest opening prices**

In [15]:
opening_prices = [day['Open'] for day in formatted_data if day['Open'] is not None]

# Calculate highest and lowest opening prices
highest_open = max(opening_prices)
lowest_open = min(opening_prices)

print(f"Highest opening price: {highest_open}")
print(f"Lowest opening price: {lowest_open}")

Highest opening price: 53.11
Lowest opening price: 34.0


**4. Largest change in on day**


In [16]:
daily_changes = [day['High'] - day['Low'] for day in formatted_data if day['High'] is not None and day['Low'] is not None]

# Find the largest change in any one day
largest_change = max(daily_changes)

print(f"Largest change in any one day: {largest_change}")

Largest change in any one day: 2.8100000000000023


**5. Largest change in two days**


In [17]:
changes_between_days = []

# Iterate through the formatted_data list starting from the second element
for i in range(1, len(formatted_data)):
    # Calculate the absolute change between consecutive days
    change = abs(formatted_data[i]['Close'] - formatted_data[i-1]['Close'])
    changes_between_days.append(change)

# Find the largest change between any two days
largest_change_between_days = max(changes_between_days)

print(f"Largest change between any two days: {largest_change_between_days}")

Largest change between any two days: 2.559999999999995


**6. average daily trading volume**

In [18]:
# Extract the trading volume for each day
trading_volumes = [day['Traded Volume'] for day in formatted_data if day['Traded Volume'] is not None]

# Calculate the average trading volume
average_trading_volume = sum(trading_volumes) / len(trading_volumes)

print(f"Average daily trading volume during the year: {average_trading_volume}")


Average daily trading volume during the year: 89124.33725490196


**7. Median trading volume during this year.**

In [19]:
def calculate_median(values):
    """
    Calculate the median of a list of values.
    Args:
    - values: A list of numbers.
    
    Returns:
    - The median value.
    """
    # sort the list
    sorted_values = sorted(values)
    n = len(sorted_values)
    
    # If the list has an odd number of elements, return the middle one.
    if n % 2 == 1:
        return sorted_values[n // 2]
    # If the list has an even number of elements, return the average of the middle two.
    else:
        return (sorted_values[n // 2 - 1] + sorted_values[n // 2]) / 2

# Extract the trading volume for each day
trading_volumes = [day['Traded Volume'] for day in formatted_data if day['Traded Volume'] is not None]

# Calculate the median trading volume
median_trading_volume = calculate_median(trading_volumes)

print(f"Median trading volume during the year: {median_trading_volume}")


Median trading volume during the year: 76286.0
